In [6]:
import numpy as np
import numpy as np
from collections import Counter
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

###### * The key to the CART algorithm is finding the optimal feature and threshold such that the Gini impurity is minimized. To do so, we try all possible splits and compute the resulting Gini impurities

In [3]:
def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

* The recursion stops when the maximum depth, a hyperparameter, is reached, or when no split can lead to two children purer than their parent

In [4]:
class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

In [5]:
class DecisionTree:
    # Defining the min_sample, max_depth , n_fetaures for split
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None
        
    # finding the number of features and root to grow the tree
    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)
        
    # predicting tarhet values 
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
                or n_labels == 1
                or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        
        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

In [7]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [8]:
data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


Accuracy: 0.9298245614035088


In [12]:
X_train[:3]

array([[1.288e+01, 1.822e+01, 8.445e+01, 4.931e+02, 1.218e-01, 1.661e-01,
        4.825e-02, 5.303e-02, 1.709e-01, 7.253e-02, 4.426e-01, 1.169e+00,
        3.176e+00, 3.437e+01, 5.273e-03, 2.329e-02, 1.405e-02, 1.244e-02,
        1.816e-02, 3.299e-03, 1.505e+01, 2.437e+01, 9.931e+01, 6.747e+02,
        1.456e-01, 2.961e-01, 1.246e-01, 1.096e-01, 2.582e-01, 8.893e-02],
       [1.113e+01, 2.244e+01, 7.149e+01, 3.784e+02, 9.566e-02, 8.194e-02,
        4.824e-02, 2.257e-02, 2.030e-01, 6.552e-02, 2.800e-01, 1.467e+00,
        1.994e+00, 1.785e+01, 3.495e-03, 3.051e-02, 3.445e-02, 1.024e-02,
        2.912e-02, 4.723e-03, 1.202e+01, 2.826e+01, 7.780e+01, 4.366e+02,
        1.087e-01, 1.782e-01, 1.564e-01, 6.413e-02, 3.169e-01, 8.032e-02],
       [1.263e+01, 2.076e+01, 8.215e+01, 4.804e+02, 9.933e-02, 1.209e-01,
        1.065e-01, 6.021e-02, 1.735e-01, 7.070e-02, 3.424e-01, 1.803e+00,
        2.711e+00, 2.048e+01, 1.291e-02, 4.042e-02, 5.101e-02, 2.295e-02,
        2.144e-02, 5.891e-03, 1.333e

In [11]:
y_train[:3]

array([1, 1, 1])

In [13]:
clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)
    
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print ("Accuracy:", acc)

Accuracy: 0.9298245614035088
